# Creación de un mapa de contaminación

Se pretende cargar la posición de las estaciones de medición de contaminantes junto con sus contaminantes y realizar una interpolación de estos contaminantes en todo un mallado entre las estaciones conocidas.

In [116]:
import numpy as np
import pandas as pd
import re
from scipy.interpolate import SmoothBivariateSpline
import matplotlib.pyplot as plt 
import pyarrow #uninstall libboost & conda install pyarrow=0.13.* -c conda-forge

In [117]:
def dms2dd(input):
    "Esta función tranforma un punto expresado en coordenadas geográficas a coordenadas UTM"
    degrees,minutes,seconds,direction=re.split('[º\´"]+',str(input))
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'S' or direction == 'W':
        dd *= -1
    return dd;

Se leeran las coordenadas de las estaciones y se filtrarán las que son de nuestro interés

In [118]:
posicion_estaciones=pd.read_excel('./data/EstacionesLocalizacion.xlsx') #Se carga el fichero

Se filtra el fichero de estaciones a la estaciones de nuestro estudio

In [119]:
Estacion=[4,8,11,35,38,39,47,48,49,50]
posicion_estaciones=posicion_estaciones[posicion_estaciones['Estacion'].isin(Estacion)]
posicion_estaciones.set_index('Estacion',inplace=True)
posicion_estaciones

,Columna1,Latitud,Longitud
Estacion,,,
47,Mendez Alavaro,"40º23´53.17""N","3º41´12.57""O"
11,Av Ramon y Cajal,"40º27´5.29""N","3º40´38.5""O"
8,Escuelas Aguirre,"40º25´17.63""N","3º40´56.22""O"
49,Retiro,"40º24´52""N","3º40´57.3""O"
50,Plaza Castilla,"40º27´56.1""N","3º41´19.48""O"
48,Paseo Castellana,"40º26´23.61""N","3º41´25.34""O"
35,Plaza del carmen,"40º25´9.15""N","3º42´11.4""O"
38,Cuatro Caminos,"40º26´43.97""N","3º42´25.64""O"
39,Barrio del pilar,"40º28´41.64""N","3º42´41.53""O"


Se convertirán los valores de Latitud y Longitud en coordenadas decimales

In [120]:
posicion_estaciones[['Latitud','Longitud']]=posicion_estaciones[['Latitud','Longitud']].applymap(lambda x: dms2dd(x))

In [121]:
posicion_estaciones.sort_values(['Latitud','Longitud'])

,Columna1,Latitud,Longitud
Estacion,,,
47,Mendez Alavaro,40.398103,3.686825
49,Retiro,40.414444,3.682583
35,Plaza del carmen,40.419208,3.703167
8,Escuelas Aguirre,40.421564,3.682283
4,Plaza de España,40.423883,3.712197
48,Paseo Castellana,40.439892,3.690372
38,Cuatro Caminos,40.445547,3.707122
11,Av Ramon y Cajal,40.451469,3.677361
50,Plaza Castilla,40.465583,3.688744


In [122]:
sortLatitud=posicion_estaciones['Latitud'].sort_values()
sortLongitud=posicion_estaciones['Longitud'].sort_values()

Se desea obtener una matriz de puntos sobre la que realizar una interpolación. La interpolación se realizará sobre los puntos conocidos, para obtener una matriz de valores contaminantes, que representará la polución en cada espacio geografico de Madrid capital

In [123]:
Latitudes=[]
Longitudes=[]
for i in range(0,(len(sortLatitud))):
    if i==9:
        lastLat=np.round(sortLatitud.iloc[i],decimals=6)
        lastLon=np.round(sortLongitud.iloc[i],decimals=6)
        Latitudes.append(lastLat)
        Longitudes.append(lastLon)
    else:
        inicioLat=sortLatitud.iloc[i]
        inicioLon=sortLongitud.iloc[i]
        finLat=sortLatitud.iloc[i+1]
        finLon=sortLongitud.iloc[i+1]
        betweenLat=np.round(np.arange(inicioLat,finLat,0.001),decimals=6)
        betweenLon=np.round(np.arange(inicioLon,finLon,0.001),decimals=6)
        Latitudes.extend(betweenLat)
        Longitudes.extend(betweenLon)

In [124]:
datos_estaciones=pd.read_csv('./data/NO2_estaciones.csv')
datos_estaciones.head()

,FECHA,Estacion4,Estacion8,Estacion11,Estacion35,Estacion38,Estacion39,Estacion47,Estacion48,Estacion49,Estacion50
0,2015-01-01 01:00:00,83.0,168.0,93.0,109.0,102.0,104.0,112.0,108.0,136.0,127.0
1,2015-01-01 02:00:00,77.0,203.0,224.0,133.0,183.0,168.0,120.0,114.0,111.0,170.0
2,2015-01-01 03:00:00,86.0,220.0,152.0,126.0,193.0,203.0,109.0,141.0,109.0,118.0
3,2015-01-01 04:00:00,65.0,169.0,151.0,112.0,122.0,147.0,87.0,145.0,99.0,126.0
4,2015-01-01 05:00:00,53.0,154.0,106.0,109.0,97.0,159.0,76.0,142.0,103.0,140.0


In [125]:
fechas=list(datos_estaciones.FECHA)

In [126]:
fechas2015=[]
fechas2016=[]
fechas2017=[]
fechas2018=[]
fechas2019=[]
fechas2020=[]
for date in fechas:
    if re.findall('2015', date):
        fechas2015.append(date)
    elif re.findall('2016', date):
        fechas2016.append(date)
    elif re.findall('2017', date):
        fechas2017.append(date)
    elif re.findall('2018', date):
        fechas2018.append(date)
    elif re.findall('2019', date):
        fechas2019.append(date)
    else:
        fechas2020.append(date)

In [127]:
ix1 = pd.MultiIndex.from_product([fechas,Latitudes])

In [128]:
mapa = pd.DataFrame(index=ix1, columns=Longitudes, dtype=np.float32)

In [ ]:
for ano in ['2016','2017','2018']:#,'2019','2020']:
    variable='fechas'+ano
    fechas=eval(variable)
    for date in fechas:
        print(date)
        x=[]
        y=[]
        Z=[]
        for i in Estacion:
            lat=posicion_estaciones.loc[i,'Latitud'].round(decimals=6)
            #print(lat)
            lon=posicion_estaciones.loc[i,'Longitud'].round(decimals=6)
            #print(lon)
            Est='Estacion'+str(i)
            sust=float(datos_estaciones[datos_estaciones['FECHA']==date][Est])
            #mapa.loc[(date,lat),lon]=sust
            x.append(lon)
            y.append(lat)
            Z.append(sust)
        #interp_spline=interpolate.interp2d(y,x,Z,kind='cubic')
        interp_spline = SmoothBivariateSpline(y,x,Z,kx=2,ky=2)
        mapa.loc[(date),:] = interp_spline(Latitudes,Longitudes)
    mapa_interp=mapa.stack().reset_index()
    mapa_interp.columns=['fecha','latitud','longitud','contaminacion']
    filename='./data/mapa_interpolado'+ano
    mapa_interp.to_feather(filename)

2016-01-01 00:00:00


/anaconda3/lib/python3.6/site-packages/scipy/interpolate/fitpack2.py:1057: UserWarning: 
The required storage space exceeds the available storage space: nxest
or nyest too small, or s too small.
The weighted least-squares spline corresponds to the current set of
knots.
  warnings.warn(message)


2016-01-01 01:00:00
2016-01-01 02:00:00
2016-01-01 03:00:00
2016-01-01 04:00:00
2016-01-01 05:00:00
2016-01-01 06:00:00
2016-01-01 07:00:00
2016-01-01 08:00:00
2016-01-01 09:00:00
2016-01-01 10:00:00
2016-01-01 11:00:00
2016-01-01 12:00:00
2016-01-01 13:00:00
2016-01-01 14:00:00
2016-01-01 15:00:00
2016-01-01 16:00:00
2016-01-01 17:00:00
2016-01-01 18:00:00
2016-01-01 19:00:00
2016-01-01 20:00:00
2016-01-01 21:00:00
2016-01-01 22:00:00
2016-01-01 23:00:00
2016-01-02 00:00:00
2016-01-02 01:00:00
2016-01-02 02:00:00
2016-01-02 03:00:00
2016-01-02 04:00:00
2016-01-02 05:00:00
2016-01-02 06:00:00
2016-01-02 07:00:00
2016-01-02 08:00:00
2016-01-02 09:00:00
2016-01-02 10:00:00
2016-01-02 11:00:00
2016-01-02 12:00:00
2016-01-02 13:00:00
2016-01-02 14:00:00
2016-01-02 15:00:00
2016-01-02 16:00:00
2016-01-02 17:00:00
2016-01-02 18:00:00
2016-01-02 19:00:00
2016-01-02 20:00:00
2016-01-02 21:00:00
2016-01-02 22:00:00
2016-01-02 23:00:00
2016-01-03 00:00:00
2016-01-03 01:00:00
2016-01-03 02:00:00


In [105]:
mapa_interp

,fecha,longitud,latitud,contaminacion
0,2016-01-01 00:00:00,40.398103,3.677361,89.803139
1,2016-01-01 00:00:00,40.398103,3.678361,85.614761
2,2016-01-01 00:00:00,40.398103,3.679361,81.465973
3,2016-01-01 00:00:00,40.398103,3.680361,77.356766
4,2016-01-01 00:00:00,40.398103,3.681361,73.287148
5,2016-01-01 00:00:00,40.398103,3.682283,69.570030
6,2016-01-01 00:00:00,40.398103,3.682583,68.367813
7,2016-01-01 00:00:00,40.398103,3.683583,64.386154
8,2016-01-01 00:00:00,40.398103,3.684583,60.444077
9,2016-01-01 00:00:00,40.398103,3.685583,56.541588


In [112]:
mapa2016=pd.read_feather('./data/mapa_interpolado2016')

In [113]:
mapa2016.set_index(['fecha','longitud','latitud'],inplace=True)

In [114]:
mapa2016.sort_index(inplace=True)

In [115]:
mapa2016.unstack('latitud')

contaminacion                          \
latitud                            3.677361    3.678361    3.679361   
fecha               longitud                                          
2016-01-01 00:00:00 40.398103     89.803139   85.614761   81.465973   
                    40.399103     89.696373   85.415283   81.186821   
                    40.400103     89.577530   85.208374   80.904510   
                    40.401103     89.446602   84.994041   80.619026   
                    40.402103     89.303596   84.772285   80.330383   
                    40.403103     89.148506   84.543106   80.038574   
                    40.404103     88.981339   84.306503   79.743599   
                    40.405103     88.802094   84.062477   79.445457   
                    40.406103     88.610764   83.811028   79.144157   
                    40.407103     88.407356   83.552147   78.839684   
                    40.408103     88.191872   83.285851   78.532051   
                    40.409103     87.964302   83.012123   78.221252   
                    40.410103     87.724655   82.730980   77.907288   
                    40.411103     87.472923   82.442406   77.590164   
                    40.412103     87.209122   82.146408   77.269867   
                    40.413103     86.933228   81.842987   76.946411   
                    40.414103     86.645264   81.532143   76.619789   
                    40.414444     86.544304   81.424446   76.507690   
                    40.415444     86.240135   81.103645   76.176826   
                    40.416444     85.923882   80.775421   75.842789   
                    40.417444     85.595558   80.439774   75.505600   
                    40.418444     85.255142   80.096703   75.165237   
                    40.419208     84.986931   79.829590   74.903069   
                    40.420208     84.625214   79.473419   74.557129   
                    40.421208     84.251411   79.109833   74.208015   
                    40.421564     84.115425   78.978600   74.082970   
                    40.422564     83.725243   78.604935   73.729568   
                    40.423564     83.322983   78.223854   73.373009   
                    40.423883     83.192123   78.100723   73.258598   
                    40.424883     82.773933   77.709846   72.897858   
...                                     ...         ...         ...   
2016-12-31 23:00:00 40.450547    185.565247  176.010101  167.206116   
                    40.451469    167.125656  159.726212  152.983917   
                    40.452469    146.058945  141.131363  136.752121   
                    40.453469    123.881798  121.565277  119.681190   
                    40.454469    100.594208  101.027954  101.771126   
                    40.455469     76.196175   79.519386   83.021927   
                    40.456469     50.687695   57.039585   63.433598   
                    40.457469     24.068771   33.588539   43.006134   
                    40.458469     -3.660593    9.166259   21.739532   
                    40.459469    -32.500401  -16.227262   -0.366203   
                    40.460469    -62.450653  -42.592022  -23.311073   
                    40.461469    -93.511345  -69.928017  -47.095078   
                    40.462469   -125.682487  -98.235260  -71.718216   
                    40.463469   -158.964066 -127.513733  -97.180489   
                    40.464469   -193.356094 -157.763443 -123.481895   
                    40.465469   -228.858566 -188.984406 -150.622437   
                    40.465583   -232.976364 -192.605270 -153.769745   
                    40.466583   -269.715851 -224.908173 -181.845078   
                    40.467583   -307.565796 -258.182343 -210.759552   
                    40.468583   -346.526184 -292.427734 -240.513168   
                    40.469583   -386.597046 -327.644348 -271.105896   
                    40.470583   -427.778320 -363.832214 -302.537781   
                    40.471583   -470.070038 -400.99133

In [109]:
mapa2016

latitud  contaminacion
fecha               longitud                          
2016-01-01 00:00:00 40.398103  3.677361      89.803139
                    40.398103  3.678361      85.614761
                    40.398103  3.679361      81.465973
                    40.398103  3.680361      77.356766
                    40.398103  3.681361      73.287148
                    40.398103  3.682283      69.570030
                    40.398103  3.682583      68.367813
                    40.398103  3.683583      64.386154
                    40.398103  3.684583      60.444077
                    40.398103  3.685583      56.541588
                    40.398103  3.686583      52.678680
                    40.398103  3.686825      51.749805
                    40.398103  3.687825      47.936066
                    40.398103  3.688744      44.466145
                    40.398103  3.689744      40.728367
                    40.398103  3.690372      38.401279
                    40.398103  3.691372      34.727947
                    40.398103  3.692372      31.094200
                    40.398103  3.693372      27.500038
                    40.398103  3.694372      23.945461
                    40.398103  3.695372      20.430470
                    40.398103  3.696372      16.955065
                    40.398103  3.697372      13.519243
                    40.398103  3.698372      10.123008
                    40.398103  3.699372       6.766358
                    40.398103  3.700372       3.449293
                    40.398103  3.701372       0.171813
                    40.398103  3.702372      -3.066082
                    40.398103  3.703167      -5.611963
                    40.398103  3.704167      -8.778802
...                                 ...            ...
2016-12-31 23:00:00 40.478233  3.685583     -41.469097
                    40.478233  3.686583      26.616102
                    40.478233  3.686825      42.385979
                    40.478233  3.687825     104.630333
                    40.478233  3.688744     157.686081
                    40.478233  3.689744     210.905823
                    40.478233  3.690372     241.923782
                    40.478233  3.691372     287.487396
                    40.478233  3.692372     328.348236
                    40.478233  3.693372     364.506317
                    40.478233  3.694372     395.961609
                    40.478233  3.695372     422.714142
                    40.478233  3.696372     444.763885
                    40.478233  3.697372     462.110870
                    40.478233  3.698372     474.755066
                    40.478233  3.699372     482.696503
                    40.478233  3.700372     485.935150
                    40.478233  3.701372     484.471039
                    40.478233  3.702372     478.304138
                    40.478233  3.703167     470.045959
                    40.478233  3.704167     455.437592
                    40.478233  3.705167     436.126434
                    40.478233  3.706167     412.112518
                    40.478233  3.707122     384.789124
                    40.478233  3.708122     351.581299
                    40.478233  3.709122     313.670685
                    40.478233  3.710122     271.057281
                    40.478233  3.711122     223.741135
                    40.478233  3.711536     202.775742
                    40.478233  3.712197     167.631195

[29381040 rows x 2 columns]

Para poder pasar los datos generados a una versión ordenada, que según Hadley Wickham, deberá realizar un "melt"

In [ ]:
mapa_interp=mapa.stack().reset_index()